# Preprocessing and Baselines

### Notebook Goals:
In this notebook, we prepare the data for modeling and train baseline models to establish a benchmark for churn predictions.

### Load Data and Initial Cleanup

### Train/Test Split

### Feature Encoding Stategy

### Baseline Models